In [3]:
%pip install rdkit
%pip install mordred
%pip install pubchempy

In [4]:
import pandas as pd
import rdkit
import matplotlib.pyplot as plt
import numpy as np
import mordred

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from gensim.models import word2vec
import seaborn as sns
from rdkit import Chem
from mordred import Calculator, descriptors
import pubchempy as pcp

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data = pd.read_csv('/content/drive/MyDrive/Алгоритмы ХИИ/new_qm9.csv')

In [17]:
data.head()

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_47998,O=CC#CC1NC1C#N,3.18840,0.81811,0.68296,5.2689,76.23,-0.2760,-0.0779,0.1980,1536.2250,0.087848,-415.556589,-415.547849,-415.546905,-415.591837,29.286
1,gdb_82240,CC1C2OC2C2CCC12,2.49891,1.84690,1.37865,1.7357,80.46,-0.2519,0.0814,0.3333,1025.3787,0.184554,-387.077518,-387.069804,-387.068860,-387.109409,31.170
2,gdb_10336,COC(C=O)C(C)O,2.35684,1.74283,1.08466,2.5701,65.19,-0.2421,-0.0289,0.2132,1101.7221,0.149467,-422.068714,-422.058690,-422.057746,-422.104380,34.081
3,gdb_116764,COC=NCC(N)C#N,4.94611,0.59499,0.55127,6.0816,79.07,-0.2514,-0.0038,0.2477,1953.8524,0.146162,-435.260337,-435.249878,-435.248933,-435.297426,35.231
4,gdb_105134,COC1(CC1C#C)C#C,2.55281,1.23863,1.00440,1.5674,83.69,-0.2402,0.0209,0.2612,1232.9278,0.131554,-384.590827,-384.581118,-384.580174,-384.625201,35.750


Generate Mordred Descriptors

In [9]:
data = pd.read_csv('/content/drive/MyDrive/Алгоритмы ХИИ/new_qm9.csv')

# Строка задает цикл в котором молекулярная форма применяется для каждой строки smiles во всем столбце SMILES
molecules = [Chem.MolFromSmiles(smiles) for smiles in data['smiles']]
calc = Calculator(descriptors)

# След строка вычисляет значение дескриптора для каждой smiles и выводит в готовый датасет
mordred_df = calc.pandas(molecules)

mordred_df.to_csv('/content/drive/MyDrive/Алгоритмы ХИИ/mordred_descriptors.csv', index=False)


100%|██████████| 20000/20000 [34:06<00:00,  9.77it/s]


In [11]:
mordred_df.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,11.385677,2.372130,4.254884,11.385677,1.265075,3.155831,...,8.773230,56.918277,120.032363,9.233259,105,7,40.0,45.0,3.472222,2.277778
1,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,11.712414,2.634284,4.950176,11.712414,1.301379,3.322713,...,9.964253,61.835066,124.088815,5.908991,75,10,58.0,76.0,2.305556,1.805556
2,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,8.720566,2.101003,4.202006,8.720566,1.090071,2.900850,...,8.124151,34.544698,118.062994,6.559055,67,8,30.0,31.0,4.722222,2.111111
3,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,10.472136,2.000000,4.000000,10.472136,1.163571,3.007971,...,7.739359,34.994205,127.074562,7.059698,110,7,32.0,31.0,4.361111,2.416667
4,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,11.278503,2.519914,4.570932,11.278503,1.253167,3.180780,...,9.372374,59.406478,120.057515,7.062207,89,10,44.0,54.0,4.173611,2.291667


 Generate RDKit Descriptors

In [13]:
def calculate_rdkit_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        descriptors = {}
        descriptors['smiles'] = smiles
        descriptors = rdkit.Chem.Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)
        return descriptors
    else:
        return None

In [14]:
rdkit_descriptors = data['smiles'].apply(calculate_rdkit_descriptors)
# mordred_descriptors = df['SMILES'].apply(calculate_mordred_descriptors)

# Создание нового датасета с дескрипторами
rdkit_df = pd.DataFrame(list(rdkit_descriptors.dropna()))
# rdkit.Chem.Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)
# mordred_df = pd.DataFrame(list(mordred_descriptors.dropna()))
display(rdkit_df)

rdkit_df.to_csv('/content/drive/MyDrive/Алгоритмы ХИИ/descriptors.csv', index=False)

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,9.647786,9.647786,0.069259,-0.155139,0.254555,29.222222,120.111,116.079,120.032363,44,...,0,0,0,0,0,0,0,0,0,0
1,5.498981,5.498981,0.696759,0.696759,0.444930,68.333333,124.183,112.087,124.088815,50,...,0,0,0,0,0,0,0,0,0,0
2,9.892546,9.892546,0.574074,-0.710648,0.511870,17.375000,118.132,108.052,118.062994,48,...,0,0,0,0,0,0,0,0,0,0
3,8.144400,8.144400,0.295972,-0.517222,0.410522,13.000000,127.147,118.075,127.074562,50,...,0,0,0,0,0,0,0,0,0,0
4,5.174722,5.174722,0.145833,-0.407986,0.461735,38.777778,120.151,112.087,120.057515,46,...,0,0,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,10.855278,10.855278,0.098380,-0.314815,0.439878,47.666667,128.127,120.063,128.047344,50,...,0,0,0,0,0,0,0,0,0,0
19996,10.233611,10.233611,0.106667,-0.258356,0.384678,52.888889,125.127,118.071,125.047678,48,...,0,0,0,0,0,0,0,0,0,0
19997,8.495972,8.495972,0.199306,-0.608009,0.454036,10.625000,109.128,102.072,109.052764,42,...,0,0,0,0,0,0,0,0,0,0
19998,9.811389,9.811389,0.050926,0.050926,0.296292,7.625000,113.120,106.064,113.058912,44,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
rdkit_df.head()

Combine 2 datasets with descriptors

In [22]:
# Объедините датафреймы по столбцам (по оси 1)
combined_df = pd.concat([rdkit_df, mordred_df], axis=1)

combined_df.to_csv('/content/drive/MyDrive/Алгоритмы ХИИ/mordred_rdkit_descriptors.csv', index=False)

In [25]:
combined_df.head(10)

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,9.647786,9.647786,0.069259,-0.155139,0.254555,29.222222,120.111,116.079,120.032363,44,...,8.773230,56.918277,120.032363,9.233259,105,7,40.0,45.0,3.472222,2.277778
1,5.498981,5.498981,0.696759,0.696759,0.444930,68.333333,124.183,112.087,124.088815,50,...,9.964253,61.835066,124.088815,5.908991,75,10,58.0,76.0,2.305556,1.805556
2,9.892546,9.892546,0.574074,-0.710648,0.511870,17.375000,118.132,108.052,118.062994,48,...,8.124151,34.544698,118.062994,6.559055,67,8,30.0,31.0,4.722222,2.111111
3,8.144400,8.144400,0.295972,-0.517222,0.410522,13.000000,127.147,118.075,127.074562,50,...,7.739359,34.994205,127.074562,7.059698,110,7,32.0,31.0,4.361111,2.416667
4,5.174722,5.174722,0.145833,-0.407986,0.461735,38.777778,120.151,112.087,120.057515,46,...,9.372374,59.406478,120.057515,7.062207,89,10,44.0,54.0,4.173611,2.291667
5,8.319259,8.319259,0.270833,0.270833,0.514507,8.888889,122.131,116.083,122.059246,46,...,8.782476,51.691682,122.059246,8.137283,84,10,42.0,48.0,4.083333,2.138889
6,10.580408,10.580408,0.146111,0.146111,0.491921,28.555556,129.159,118.071,129.078979,52,...,8.590258,37.289972,129.078979,6.453949,90,9,40.0,43.0,3.472222,2.166667
7,8.884028,8.884028,0.002894,0.002894,0.541844,67.000000,126.155,116.075,126.068080,50,...,10.394610,64.379700,126.068080,6.635162,77,7,60.0,82.0,2.395833,1.888889
8,9.352130,9.352130,0.087963,-0.187500,0.511467,63.666667,128.171,116.075,128.083730,52,...,10.063095,57.549571,128.083730,6.099225,74,10,54.0,70.0,3.895833,1.847222
9,5.503472,5.503472,0.008102,0.008102,0.489257,63.666667,128.171,116.075,128.083730,52,...,9.947552,62.484101,128.083730,6.099225,76,8,56.0,71.0,3.986111,1.729167


In [20]:
# Объедините датафреймы по столбцам (по оси 1)
united_df = pd.concat([data, rdkit_df, mordred_df], axis=1)

united_df.to_csv('/content/drive/MyDrive/Алгоритмы ХИИ/combined_dataset.csv', index=False)

In [24]:
united_df.head(10)

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,gdb_47998,O=CC#CC1NC1C#N,3.18840,0.81811,0.68296,5.2689,76.23,-0.2760,-0.0779,0.1980,...,8.773230,56.918277,120.032363,9.233259,105,7,40.0,45.0,3.472222,2.277778
1,gdb_82240,CC1C2OC2C2CCC12,2.49891,1.84690,1.37865,1.7357,80.46,-0.2519,0.0814,0.3333,...,9.964253,61.835066,124.088815,5.908991,75,10,58.0,76.0,2.305556,1.805556
2,gdb_10336,COC(C=O)C(C)O,2.35684,1.74283,1.08466,2.5701,65.19,-0.2421,-0.0289,0.2132,...,8.124151,34.544698,118.062994,6.559055,67,8,30.0,31.0,4.722222,2.111111
3,gdb_116764,COC=NCC(N)C#N,4.94611,0.59499,0.55127,6.0816,79.07,-0.2514,-0.0038,0.2477,...,7.739359,34.994205,127.074562,7.059698,110,7,32.0,31.0,4.361111,2.416667
4,gdb_105134,COC1(CC1C#C)C#C,2.55281,1.23863,1.00440,1.5674,83.69,-0.2402,0.0209,0.2612,...,9.372374,59.406478,120.057515,7.062207,89,10,44.0,54.0,4.173611,2.291667
5,gdb_125012,Cc1c(c(n[nH]1)C#N)N,3.36988,1.28989,0.93978,6.5686,76.43,-0.2184,-0.0218,0.1967,...,8.782476,51.691682,122.059246,8.137283,84,10,42.0,48.0,4.083333,2.138889
6,gdb_115234,OCC1CCC(=O)CN1,3.73014,0.97687,0.82044,1.2026,75.71,-0.2260,-0.0240,0.2019,...,8.590258,37.289972,129.078979,6.453949,90,9,40.0,43.0,3.472222,2.166667
7,gdb_108150,OCC12OC3CCC1C23,3.44265,1.31210,1.25365,2.1488,74.05,-0.2287,0.0659,0.2945,...,10.394610,64.379700,126.068080,6.635162,77,7,60.0,82.0,2.395833,1.888889
8,gdb_83341,CC1C2OCC1(C)C2O,2.07997,1.64142,1.50155,0.9353,77.72,-0.2291,0.0663,0.2954,...,10.063095,57.549571,128.083730,6.099225,74,10,54.0,70.0,3.895833,1.847222
9,gdb_89384,CC1OC2(C)CC2(C)O1,2.53679,1.51918,1.18388,1.1838,78.86,-0.2337,0.0842,0.3179,...,9.947552,62.484101,128.083730,6.099225,76,8,56.0,71.0,3.986111,1.729167
